## Import Libraries and API Key

In [1]:
import os
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SequentialChain, ConversationChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from secret_key import openai_key

os.environ['OPENAI_API_KEY'] = openai_key

## Choose the model to use.

In [2]:
# Initialize the LLM with the specific model
llm = OpenAI(temperature=0.5)

c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## Define the prompt template.

In [3]:
# Define the prompt template
poem_start_prompt = PromptTemplate(
    input_variables=["word"],
    template="Write a poem that describes the word '{word}' in a simple way, to teach a child. Write only one stanza.",

)

# Define the prompt template for adding a stanza to the poem
poem_complete_prompt = PromptTemplate(
    input_variables=["poem_start"],
    template="Complete the poem, that starts as :\n{poem_start}"
)

## Build the chain.

In [4]:
# Build a chain by specifying the LLM and the prompt template
poem_start_chain = LLMChain(llm=llm, prompt=poem_start_prompt, output_key="poem_start")
poem_complete_chain = LLMChain(llm=llm, prompt=poem_complete_prompt, output_key="poem")

# Build a sequential chain by specifying the chains
chain = SequentialChain(
    chains=[poem_start_chain, poem_complete_chain],
    input_variables=["word"],
    output_variables=["poem"],
    verbose=True  # To see the intermediate steps
)

# Run the chain with a specific word
response = chain({"word": "dog"})

print(response)

c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
{'word': 'dog', 'poem': " \n\nThey greet us with excitement and joy,\nTheir love and loyalty we can't destroy,\nThey cuddle close when we're feeling down,\nAnd chase away our every frown.\n\nTheir eyes are filled with pure devotion,\nTheir hearts full of endless emotion,\nThey're always there to lend an ear,\nAnd wipe away every single tear.\n\nThrough the good times and the bad,\nOur furry friends are all we've had,\nThey're more than just a pet, you see,\nThey're family, as loyal as can be.\n\nSo here's to our furry companions,\nWho bring us love in endless fashions,\nThey fill our lives with so much glee,\nA dog's love is truly meant to be."}


In [5]:
print(response["poem"])

 

They greet us with excitement and joy,
Their love and loyalty we can't destroy,
They cuddle close when we're feeling down,
And chase away our every frown.

Their eyes are filled with pure devotion,
Their hearts full of endless emotion,
They're always there to lend an ear,
And wipe away every single tear.

Through the good times and the bad,
Our furry friends are all we've had,
They're more than just a pet, you see,
They're family, as loyal as can be.

So here's to our furry companions,
Who bring us love in endless fashions,
They fill our lives with so much glee,
A dog's love is truly meant to be.


## Load Tools to Use

In [6]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Initialize the agent
# The agent type is ZERO_SHOT_REACT_DESCRIPTION, this means that the agent will deside to use any tool to generate the output if needed.
agent = initialize_agent(
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    tools=tools,
    verbose=True # To see the intermediate steps
)

agent.run("When was the first computer invented? Add 50 to the answer.")



> Entering new AgentExecutor chain...


c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Users\Furkan Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 We need to find the date of the first computer and then add 50 to it.
Action: wikipedia
Action Input: First computer
Observation: Page: History of computing hardware
Summary: The history of computing hardware covers the developments from early simple devices to aid calculation to modern day computers.
The first aids to computation were purely mechanical devices which required the operator to set up the initial values of an elementary arithmetic operation, then manipulate the device to obtain the result. Later, computers represented numbers in a continuous form (e.g. distance along a scale, rotation of a shaft, or a voltage). Numbers could also be represented in the form of digits, automatically manipulated by a mechanism. Although this approach generally required more complex mechanisms, it greatly increased the precision of results. The development of transistor technology and then the integrated circuit chip led to a series of breakthroughs, starting with transistor computers and th

'1990'

## Define Memory

Chains do not have memory by default.

In [7]:
type(chain.memory)

NoneType

In [8]:
# Create a memory object
memory = ConversationBufferMemory()

# Add the memory to the chain
chain = SequentialChain(
    chains=[poem_start_chain, poem_complete_chain],
    input_variables=["word"],
    output_variables=["poem"],
    verbose=True,
    memory=memory
)

poem = chain({"word": "dog"})

print(poem)



> Entering new SequentialChain chain...

> Finished chain.
{'word': 'dog', 'history': '', 'poem': "\n\nHe'll chase his tail with endless glee,\nAnd fetch a stick with boundless speed.\nHe'll cuddle up and let you be,\nThe one he loves, the one he needs.\n\nA dog is more than just a pet,\nHe's a companion, a faithful mate.\nHe'll never judge, he'll never fret,\nJust happy to be by your side, it's fate.\n\nHe'll bark at strangers, protect your home,\nBut snuggle close when you're feeling low.\nHe'll never leave you feeling alone,\nHis love for you will always show.\n\nA dog is a furry friend indeed,\nWith a heart of gold and eyes that shine.\nHe'll be there for you in times of need,\nThis loyal companion, this friend of mine."}


We now have a memory that keeps track of the prompt-response pairs.

In [9]:
print(chain.memory.buffer)

Human: dog
AI: 

He'll chase his tail with endless glee,
And fetch a stick with boundless speed.
He'll cuddle up and let you be,
The one he loves, the one he needs.

A dog is more than just a pet,
He's a companion, a faithful mate.
He'll never judge, he'll never fret,
Just happy to be by your side, it's fate.

He'll bark at strangers, protect your home,
But snuggle close when you're feeling low.
He'll never leave you feeling alone,
His love for you will always show.

A dog is a furry friend indeed,
With a heart of gold and eyes that shine.
He'll be there for you in times of need,
This loyal companion, this friend of mine.


In [10]:
# Create a conversation chain without memory
convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


We are able to see the system prompt that includes the history that ConverSationChain keeps by default.

In [11]:
convo.run("Which country won the 2018 FIFA World Cup?")

' France won the 2018 FIFA World Cup. They defeated Croatia 4-2 in the final match, which was held in Moscow, Russia. This was their second World Cup victory, with their first win being in 1998. The French team was led by coach Didier Deschamps and star players such as Kylian Mbappé, Antoine Griezmann, and Paul Pogba. They had a strong performance throughout the tournament, winning six matches and drawing one.'

In [12]:
convo.run("Who scored the goals in the final match?")

" In the final match, France's goals were scored by Antoine Griezmann, Paul Pogba, and Kylian Mbappé. Griezmann scored a penalty kick in the 38th minute, while Pogba scored in the 59th minute and Mbappé in the 65th minute. Croatia's goals were scored by Ivan Perišić and Mario Mandžukić."

In [13]:
print(convo.memory.buffer)

Human: Which country won the 2018 FIFA World Cup?
AI:  France won the 2018 FIFA World Cup. They defeated Croatia 4-2 in the final match, which was held in Moscow, Russia. This was their second World Cup victory, with their first win being in 1998. The French team was led by coach Didier Deschamps and star players such as Kylian Mbappé, Antoine Griezmann, and Paul Pogba. They had a strong performance throughout the tournament, winning six matches and drawing one.
Human: Who scored the goals in the final match?
AI:  In the final match, France's goals were scored by Antoine Griezmann, Paul Pogba, and Kylian Mbappé. Griezmann scored a penalty kick in the 38th minute, while Pogba scored in the 59th minute and Mbappé in the 65th minute. Croatia's goals were scored by Ivan Perišić and Mario Mandžukić.


Every single time we prompt the model, we will pass the memory to it.\
This might be costly therefore we need to limit the memory size or the number of prompt-response pairs to keep in memory.\
To keep the memory size limited we can use a windowed memory.

In [14]:
# k=1 means that the memory will store the last 1 conversation.
memory = ConversationBufferWindowMemory(k=1)

# Create a chain with the limited memory
chain = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)

chain.run("Which country won the 2018 FIFA World Cup?")

" France won the 2018 FIFA World Cup. They defeated Croatia 4-2 in the final match, which took place in Moscow, Russia on July 15, 2018. France's victory was their second World Cup title, with their first being in 1998. The tournament featured 32 teams and was held in 11 cities across Russia. Did you know that the World Cup is the most widely viewed sporting event in the world, with an estimated 3.4 billion people tuning in to watch the final?"

In [15]:
chain.run("What's 1+1?")

' The answer to 1+1 is 2. This is a basic mathematical equation that represents adding one unit to another unit. The concept of addition is a fundamental mathematical operation that is used in various fields such as science, engineering, and economics. Did you know that the concept of addition dates back to ancient civilizations, such as the Egyptians and Babylonians? They used different methods, such as counting on fingers and using counting boards, to perform addition calculations.'

In [16]:
chain.run("Who scored the goals in the final match?")

' I do not have enough contextual information to accurately answer this question. Do you have any specific details about the final match, such as the teams playing, the date, or the location? With more information, I can provide a more accurate answer.'

Since our memory is limited to the last one prompt-response pair, llm does not have the information about the previous prompt\
we have specified the football match therefore it's having trouble to generate a response.